In [17]:
%pip install --upgrade pip 
%pip install sqlalchemy
%pip install sqlalchemy_utils
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached psycopg2-2.9.9-cp39-cp39-win_amd64.whl.metadata (4.6 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.2 MB 1.5 MB/s eta 0:00:01
   --------- ------------------------------ 0.3/1.2 MB 2.9 MB/s eta 0:00:01
   -------------------- ------------------- 0.6/1.2 MB 4.3 MB/s eta 0:00:01
   -------------------------------------- - 1.1/1.2 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
from enum import Enum
from typing import List
from uuid import UUID
from datetime import datetime

from sqlalchemy import (
    create_engine,
    inspect,
    DateTime,
    ForeignKey,
    UniqueConstraint,
    types
)
from sqlalchemy.orm import Mapped, mapped_column, relationship, validates, declarative_base
from sqlalchemy_utils import database_exists, create_database

In [28]:
Base = declarative_base()



class LikeDislike(Enum):
    like = 1
    dislike = -1


class Review(Base):
    __tablename__ = "review"
    __table_args__ = (
        UniqueConstraint('user_id', 'movie_id', name='unique_review_user_movie'),
    )

    id: Mapped[UUID] = mapped_column(primary_key=True)

    user_id: Mapped[UUID]
    movie_id: Mapped[UUID] = mapped_column(index=True)

    score: Mapped[int] = mapped_column(nullable=False, comment="Оценка отзыва")
    text: Mapped[str] = mapped_column(nullable=False, comment="Текст отзыва")
    review_dt: Mapped[datetime] = mapped_column(nullable=False)

    reviews_ratings: Mapped[List['UserReviewRating']] = relationship(
        back_populates="review", cascade='all, delete')
    
    @validates('score')
    def validate_score(self, key, value):
        if 0 <= value <= 10:
            return value
        raise ValueError("Value should be between 0 and 10")



class Bookmark(Base):
    __tablename__ = "bookmark"
    __table_args__ = (
        UniqueConstraint('user_id', 'movie_id', name='unique_bookmark_user_movie'),
    )

    id: Mapped[UUID] = mapped_column(primary_key=True)

    user_id: Mapped[UUID] = mapped_column(index=True)
    movie_id: Mapped[UUID] = mapped_column(types.Uuid)


class UserReviewRating(Base):
    __tablename__ = "user_review_rating"
    __table_args__ = (
        UniqueConstraint('user_id', 'review_id', name='unique_user_review_rating'),
    )
    id: Mapped[UUID] = mapped_column(primary_key=True)

    user_id: Mapped[int] = mapped_column(index=True)

    review: Mapped["Review"] = relationship(back_populates="reviews_ratings")
    review_id: Mapped[UUID] = mapped_column(ForeignKey("review.id"))

    score: Mapped[LikeDislike]



In [29]:

user = 'bench_user'
password = 'bench_pass'
host = 'localhost'
port = 5434
db = 'bench_db'

pg_url = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'
if not database_exists(pg_url):
    print('No db')
    create_database(pg_url)
    print('db created')

engine = create_engine(pg_url)

inspector = inspect(engine)
db_tables = inspector.get_table_names()

if not db_tables:
    print('Моделей нет в БД')
    Base.metadata.create_all(bind=engine)
    print('Модели добавлены в БД')
else:
    print('Моделей уже есть в БД')

Моделей нет в БД


ProgrammingError: (psycopg2.errors.DuplicateTable) relation "unique_user_movie" already exists

[SQL: 
CREATE TABLE bookmark (
	id UUID NOT NULL, 
	user_id UUID NOT NULL, 
	movie_id UUID NOT NULL, 
	PRIMARY KEY (id), 
	CONSTRAINT unique_user_movie UNIQUE (user_id, movie_id)
)

]
(Background on this error at: https://sqlalche.me/e/20/f405)